# Exploring Scalability in C++ Parallel STL Implementations
This jupyter notebook will reserve and setup the node to reproduce the experiments in the paper "Exploring Scalability in C++ Parallel STL Implementations".
After the execution of this notebook, the user should be able to `ssh` into the system and execute there the scripts to reproduce the results of the paper.

In [ ]:
import chi
PROJECT_NAME="YOUR_PROJECT_NAME"
chi.use_site('CHI@TACC')
chi.set("project_name",PROJECT_NAME)

In [ ]:
from chi import server
from os import environ

# get username, but we can't use "_" in some of the next steps, so replace with "-"
username = environ.get('USER').replace("_", "-")

IMAGE_NAME = "ubuntu22-pstl"
IMAGE_ID = "b9aee79d-7f59-4469-a290-f479b48808bf"
MASTER_NAME = f"icpp-pstl"
NETWORK_NAME = "sharednet1"

FLAVOR_NAME = "baremetal"
NODE_TYPE = "compute_zen3"

print(f'Image: {IMAGE_NAME}')
print(f'Master: {MASTER_NAME}')
print(f'Network: {NETWORK_NAME}')
print(f'Flavor: {FLAVOR_NAME}')
print(f'Node type: {NODE_TYPE}')

SERVER_COUNT = 1

In [ ]:
# Create a reservation for your nodes
from chi import lease

reservations = []
lease.add_node_reservation(
    reservations, count=SERVER_COUNT, node_type=NODE_TYPE)

start_date, end_date = lease.lease_duration(days=1) # hours=2

lease_info = lease.create_lease(f"{username}-lease",
                                reservations=reservations,
                                start_date="now",
                                end_date=end_date
                                )
lease_id = lease_info.get("id")
active_lease_info = lease.wait_for_active(lease_id)

In [ ]:
reservation_id = lease.get_node_reservation(lease_id)
display(reservation_id)

In [ ]:
print('Creating server...')
computing_node = server.create_server(
    server_name=MASTER_NAME,
    flavor_name=FLAVOR_NAME,
    image_name=IMAGE_NAME,
    image_id=IMAGE_ID,
    network_name=NETWORK_NAME,
    reservation_id=reservation_id,
)
print('Server created')
print('Waiting for node to be active...')
server.wait_for_active(computing_node.id)
print('Node is active')

In [ ]:
# associate the ssh security group with our computing node
computing_fip = chi.server.associate_floating_ip(computing_node.id)

In [ ]:
# wait for the server + sshd to become active. Especially with baremetal, it can take some time to boot
server.wait_for_tcp(computing_fip, 22)
print(f"ssh cc@{computing_fip}")